In [39]:
!pip install --upgrade "httpx<1.0.0,>=0.28.1"

In [40]:
!pip install xrpl-py --quiet

In [41]:
import xrpl
import pkg_resources
print("xrpl-py:", pkg_resources.get_distribution("xrpl-py").version)


xrpl-py: 4.3.0


In [42]:
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.wallet import Wallet
from xrpl.models.transactions import Payment
from xrpl.asyncio.transaction import autofill, sign, submit_and_wait
from xrpl.models.requests import AccountInfo, Tx

JSON_RPC_URL = "https://s.altnet.rippletest.net:51234"
client = AsyncJsonRpcClient(JSON_RPC_URL)

ALICE_SEED  = "sEdS5dwCvKMCCrHKo1siF3A6jbTR5WU"       # faucet secret
alice = Wallet.from_seed(ALICE_SEED)

async def balance_drops(addr: str) -> int:
    req = AccountInfo(account=addr, ledger_index="validated", strict=True)
    res = await client.request(req)
    return int(res.result["account_data"]["Balance"])

print("Before -> Alice:", await balance_drops(alice.classic_address))

# Generate a new wallet for Bob using the testnet faucet
from xrpl.asyncio.wallet import generate_faucet_wallet

BOB_WALLET = await generate_faucet_wallet(client, debug=True)
BOB_ADDRESS = BOB_WALLET.classic_address

print("New Bob Address:", BOB_ADDRESS)
print("New Bob Seed:", BOB_WALLET.seed)

print("After -> Alice:", await balance_drops(alice.classic_address),
      "| Bob:", await balance_drops(BOB_ADDRESS))

Before -> Alice: 4999950
Attempting to fund address rJqVJxBws9qJh4NKYQVdhbKjgyGE7Ky64g
Faucet fund successful.
New Bob Address: rJqVJxBws9qJh4NKYQVdhbKjgyGE7Ky64g
New Bob Seed: sEdVo1drGoEfT2N6AW8sqfHRQJewsrj
After -> Alice: 4999950 | Bob: 10000000


In [43]:
# Build payment
print("Alice's address:", alice.classic_address)
print("Bob's address:", BOB_ADDRESS)
tx = Payment(
    account=alice.classic_address,
    destination=BOB_ADDRESS, # Use the updated BOB_ADDRESS
    amount="1000000"  # 1 XRP in drops
)

# Async -> autofill, sync -> sign, async -> submit_and_wait
tx = await autofill(tx, client)     # await (network call)
signed = sign(tx, alice)            # NO await (pure local signing)
final = await submit_and_wait(signed, client)  # await (network/validation)

tx_hash = signed.get_hash()
print("TX Hash:", tx_hash)

Alice's address: rBBuyBHzNz8zTNH9g76Zi5mty4BCDyYEaS
Bob's address: rJqVJxBws9qJh4NKYQVdhbKjgyGE7Ky64g
TX Hash: AB34CA81C981F180DEDDA06FB553AB6ABE0A3F5E20464E3059BD4387448E60A3


In [44]:
import xrpl.asyncio.wallet
print(dir(xrpl.asyncio.wallet))

['XRPLFaucetException', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'generate_faucet_wallet', 'wallet_generation']


In [45]:
# Generate a new wallet for Bob using the testnet faucet
from xrpl.asyncio.wallet import generate_faucet_wallet

BOB_WALLET = await generate_faucet_wallet(client, debug=True)
BOB_ADDRESS = BOB_WALLET.classic_address

print("New Bob Address:", BOB_ADDRESS)
print("New Bob Seed:", BOB_WALLET.seed)

Attempting to fund address rQ9TDp4ng6aQev95CanZzh5A89AYqSKjoE
Faucet fund successful.
New Bob Address: rQ9TDp4ng6aQev95CanZzh5A89AYqSKjoE
New Bob Seed: sEdTPdfAkyWmHd41uGCRryDmgm3MCGt


In [46]:
from xrpl.models.requests import Tx

# If tx_hash variable isn’t in scope, paste it:
# tx_hash = "BA000472497676B6978E7D253F82624FA9A4290513F8911A7ED155F6BF0D0411"

tx_info = (await client.request(Tx(transaction=tx_hash))).result
print("Validated:", tx_info.get("validated"))
print("Engine Result:", tx_info.get("meta", {}).get("TransactionResult"))
print("Fee (drops):", tx_info.get("Fee"))
print("Destination:", tx_info.get("Destination"))
print("Amount (drops):", tx_info.get("Amount"))


Validated: True
Engine Result: tesSUCCESS
Fee (drops): None
Destination: None
Amount (drops): None


In [47]:
print("After -> Alice:", await balance_drops(alice.classic_address),
      "| Bob:", await balance_drops(BOB_ADDRESS))

After -> Alice: 3999940 | Bob: 10000000


In [48]:
from xrpl.models.transactions import Payment, Memo
import binascii

# Example metadata
DEST_TAG = 2025
note = "AIML Lab: XRPL payment"

# XRPL requires memo data to be hex-encoded
note_hex = binascii.hexlify(note.encode("utf-8")).decode("utf-8")

# Build Payment with memo + destination tag
tx = Payment(
    account=alice.classic_address,
    destination=BOB_ADDRESS,
    amount="1000000",        # 1 XRP
    destination_tag=DEST_TAG,
    memos=[Memo(memo_data=note_hex)],
)

# Autofill, sign, and submit
tx = await autofill(tx, client)
signed = sign(tx, alice)        # sync
final = await submit_and_wait(signed, client)

print("TX Hash (memo+tag):", signed.get_hash())
EC8B93CCCBA1C0EB8FBC7EC7DE74E7948CABD32CB2A7AF41A25B878F9ADF95F7

TX Hash (memo+tag): EC8B93CCCBA1C0EB8FBC7EC7DE74E7948CABD32CB2A7AF41A25B878F9ADF95F7
